# IntrinClock

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.IntrinClock)

class IntrinClock(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies an anti-logarithmic linear transformation to a PyTorch tensor.
        """
        adult_age = 20

        # Create a mask for negative and non-negative values
        mask_negative = x < 0
        mask_non_negative = ~mask_negative

        # Initialize the result tensor
        age_tensor = torch.empty_like(x)

        # Exponential transformation for negative values
        age_tensor[mask_negative] = (1 + adult_age) * torch.exp(x[mask_negative]) - 1

        # Linear transformation for non-negative values
        age_tensor[mask_non_negative] = (1 + adult_age) * x[mask_non_negative] + adult_age

        return age_tensor



In [3]:
model = pya.models.IntrinClock()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'intrinclock'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2024
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Tomusiak, Alan, et al. \"Development of an epigenetic clock resistant to changes in immune cell composition.\" Communications Biology 7.1 (2024): 934."
model.metadata["doi"] = 'https://doi.org/10.1038/s42003-024-06609-4'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
coefficient_dict = {
    'cg00054496': 0.629765,
    'cg00055555': 0.000478,
    'cg00088042': 0.542504,
    'cg00107606': -0.602358,
    'cg00359604': -0.207408,
    'cg00448707': 0.101321,
    'cg00454305': 0.214177,
    'cg00480331': 0.279553,
    'cg00491255': 0.235923,
    'cg00579423': -0.023261,
    'cg00583733': -0.187858,
    'cg00588393': 0.306040,
    'cg00590036': 0.946985,
    'cg00593462': 0.339070,
    'cg00716277': -0.117843,
    'cg00734800': -0.282424,
    'cg00751156': 0.895214,
    'cg00753885': -0.463940,
    'cg00790928': -0.333871,
    'cg00792123': -0.455670,
    'cg00796360': -0.040682,
    'cg01028142': 0.248537,
    'cg01080924': -0.063070,
    'cg01131866': -0.143417,
    'cg01153342': -0.434237,
    'cg01176694': -0.164361,
    'cg01260146': -0.685730,
    'cg01410876': 0.000990,
    'cg01426818': 0.090433,
    'cg01447660': -0.377915,
    'cg01483824': 0.001925,
    'cg01632288': 0.020347,
    'cg01676322': -0.008812,
    'cg01723606': 0.150160,
    'cg01747664': -0.195955,
    'cg01748892': 0.291528,
    'cg01773685': 0.108342,
    'cg01945641': 0.077529,
    'cg01948062': -0.243899,
    'cg02018277': -0.101781,
    'cg02071825': 0.663272,
    'cg02118627': -0.015532,
    'cg02273647': 0.098363,
    'cg02315513': 0.163021,
    'cg02372745': 0.844674,
    'cg02605776': 0.102264,
    'cg02632185': 0.219644,
    'cg02677192': 0.075650,
    'cg02695349': 0.014691,
    'cg02741548': -0.230018,
    'cg02773698': 0.154705,
    'cg02798181': -0.012452,
    'cg02813298': -0.021371,
    'cg02821342': -0.835542,
    'cg02869559': -0.178805,
    'cg02905830': -0.065400,
    'cg03039990': -0.112718,
    'cg03065311': 0.144953,
    'cg03080043': 0.027251,
    'cg03260211': 0.219926,
    'cg03292213': -0.095841,
    'cg03331387': 0.009966,
    'cg03404662': 0.240773,
    'cg04035728': -0.020864,
    'cg04193160': 0.189600,
    'cg04232649': -0.303726,
    'cg04425110': 0.039951,
    'cg04435719': -0.424840,
    'cg04461859': 0.035512,
    'cg04508804': 0.017993,
    'cg04568797': -0.162114,
    'cg04596060': -0.480476,
    'cg04768459': -0.195784,
    'cg04847146': -0.088714,
    'cg04875128': 0.333810,
    'cg04897804': 0.095390,
    'cg04946721': 0.132790,
    'cg04982874': 0.030834,
    'cg05147654': -0.012265,
    'cg05324516': -0.243302,
    'cg05396044': 1.052750,
    'cg05442902': -0.380687,
    'cg05473871': -0.195023,
    'cg05502081': -0.317073,
    'cg05527430': 0.228954,
    'cg05638500': 0.404251,
    'cg05675373': 0.052474,
    'cg05714496': -0.140482,
    'cg05760722': -0.169668,
    'cg05782445': 0.243627,
    'cg05790551': 0.689393,
    'cg05823563': -0.221066,
    'cg05826458': 0.063505,
    'cg05896926': 0.111656,
    'cg05915866': -0.233932,
    'cg06208270': -0.161155,
    'cg06279276': 0.001596,
    'cg06339542': -0.105458,
    'cg06375761': -0.013404,
    'cg06385324': 1.149900,
    'cg06460983': 0.242894,
    'cg06462666': -0.092600,
    'cg06526721': -0.016389,
    'cg06568490': 0.250606,
    'cg06685111': -0.449694,
    'cg06739520': 0.608522,
    'cg06766273': -0.442990,
    'cg06785746': 0.062742,
    'cg06836406': 0.176486,
    'cg06889422': -0.063114,
    'cg06943141': 0.077398,
    'cg06965409': -0.448577,
    'cg06975196': -0.177409,
    'cg07077115': -0.095823,
    'cg07082267': -0.237335,
    'cg07158339': -0.076210,
    'cg07292235': 0.012162,
    'cg07337598': 0.059696,
    'cg07477282': 1.446001,
    'cg07589899': 0.563002,
    'cg07739179': 0.034575,
    'cg07759394': -0.350554,
    'cg07955995': 0.287813,
    'cg08033031': -0.210983,
    'cg08046044': 0.234338,
    'cg08074084': 0.270753,
    'cg08097417': 2.583904,
    'cg08169949': 0.730853,
    'cg08279008': -0.334302,
    'cg08296037': -0.057226,
    'cg08356445': 0.145932,
    'cg08357125': -0.177629,
    'cg08482080': -0.361281,
    'cg08521677': -0.385882,
    'cg08857478': -0.231122,
    'cg08993878': -0.083171,
    'cg09036297': 0.219183,
    'cg09105193': 0.594248,
    'cg09214983': -0.150207,
    'cg09278098': -0.081549,
    'cg09281539': -0.130365,
    'cg09338875': -0.653133,
    'cg09406384': -0.873546,
    'cg09553982': 0.029792,
    'cg09624130': -0.162465,
    'cg09661809': 0.320360,
    'cg09692396': -0.493154,
    'cg09693106': -0.027079,
    'cg09809672': -0.153914,
    'cg09841001': -0.081456,
    'cg09846458': -0.189517,
    'cg09910998': -0.109396,
    'cg09962925': 0.512151,
    'cg09971754': -0.199292,
    'cg10064339': -0.140178,
    'cg10070101': -0.089134,
    'cg10118784': 0.281102,
    'cg10163222': -0.034615,
    'cg10192736': -0.270267,
    'cg10307580': -0.143452,
    'cg10315903': 0.051591,
    'cg10373645': -0.153072,
    'cg10381888': 0.196570,
    'cg10523019': 0.159604,
    'cg10552698': -0.376593,
    'cg10561067': -0.158230,
    'cg10596537': -0.055959,
    'cg10788210': 2.527910,
    'cg10838157': -0.176249,
    'cg10858686': -0.077274,
    'cg10933003': 0.163029,
    'cg10951117': 0.025426,
    'cg11051055': 0.201423,
    'cg11059561': 0.318553,
    'cg11098525': 0.337549,
    'cg11109027': -0.805256,
    'cg11126313': 0.209309,
    'cg11176990': 0.359544,
    'cg11401866': 0.105506,
    'cg11431402': 0.017453,
    'cg11527930': 0.085740,
    'cg11539664': 0.242090,
    'cg11539674': 0.085122,
    'cg11582617': -0.079856,
    'cg11789185': 0.337167,
    'cg11791577': -0.114446,
    'cg11825926': -0.271932,
    'cg11989576': 0.111367,
    'cg12309479': -0.205215,
    'cg12340166': 0.071403,
    'cg12366582': -0.686739,
    'cg12397924': -0.112834,
    'cg12497786': -0.103957,
    'cg12497883': 0.062538,
    'cg12709970': -0.600442,
    'cg12791555': 0.126912,
    'cg12810523': -0.133198,
    'cg12848614': -0.199842,
    'cg12850242': 0.042669,
    'cg12924510': -0.014322,
    'cg12948621': 0.167489,
    'cg12981137': 0.088587,
    'cg13085980': -0.065387,
    'cg13136596': -0.148944,
    'cg13204699': -0.071927,
    'cg13246426': 0.252035,
    'cg13251484': -0.454989,
    'cg13365340': -0.184025,
    'cg13443627': 0.059344,
    'cg13494498': 0.244286,
    'cg13575298': 0.291580,
    'cg13683374': -0.104213,
    'cg13697378': 0.083463,
    'cg13706079': 0.108472,
    'cg13720865': 0.041302,
    'cg13730743': -0.641688,
    'cg13785883': 0.945837,
    'cg13844899': -0.193918,
    'cg13854874': 0.079520,
    'cg13935689': 0.246708,
    'cg13983063': -0.192778,
    'cg14003022': -0.005578,
    'cg14034476': -0.103989,
    'cg14074174': -0.427476,
    'cg14140060': 0.032882,
    'cg14291650': -0.102791,
    'cg14295611': -0.297499,
    'cg14305711': -0.015526,
    'cg14331163': -0.149986,
    'cg14362004': 0.415994,
    'cg14424579': 1.327706,
    'cg14627089': 0.180821,
    'cg14782206': 0.119040,
    'cg14815005': -0.513939,
    'cg14829814': -0.064138,
    'cg14884160': -0.384053,
    'cg15118606': 0.170109,
    'cg15128510': 0.073551,
    'cg15159987': 0.321430,
    'cg15178202': -0.194715,
    'cg15208197': 0.132881,
    'cg15363134': -0.074360,
    'cg15548613': 0.082398,
    'cg15586420': -0.088739,
    'cg15820033': -0.028047,
    'cg15829826': -0.696270,
    'cg15851799': -0.076108,
    'cg15951188': 0.114012,
    'cg16026522': 0.182903,
    'cg16136840': -0.072940,
    'cg16241714': 0.901287,
    'cg16331674': 0.483335,
    'cg16370875': 0.391429,
    'cg16478145': 0.075640,
    'cg16485682': 0.127591,
    'cg16639637': -0.052944,
    'cg16703882': 0.525210,
    'cg16742481': -0.272142,
    'cg16750275': -0.189936,
    'cg16837441': -0.029999,
    'cg16932827': -0.223159,
    'cg17121412': -0.100472,
    'cg17152981': 0.168534,
    'cg17317023': 0.116863,
    'cg17713912': 0.124633,
    'cg17758721': 0.738194,
    'cg17796960': 0.037607,
    'cg17802633': -0.145015,
    'cg17804348': 0.157616,
    'cg17892169': -0.090332,
    'cg18087943': -0.122351,
    'cg18113084': -0.643732,
    'cg18147606': -0.241852,
    'cg18153869': 0.132170,
    'cg18336453': 0.275552,
    'cg18382305': -0.146864,
    'cg18400181': -0.476893,
    'cg18417423': 0.132226,
    'cg18424208': 0.104091,
    'cg18575740': 0.060317,
    'cg18601714': 0.110121,
    'cg18635432': 0.028141,
    'cg18644787': -0.756996,
    'cg18781966': -0.193718,
    'cg18832152': -0.596254,
    'cg18847227': 0.028334,
    'cg18881501': -0.089093,
    'cg18933331': -0.388073,
    'cg19021197': 0.229413,
    'cg19065177': 0.101292,
    'cg19216851': 0.131758,
    'cg19283196': 0.175860,
    'cg19283806': -0.281685,
    'cg19590421': -0.065829,
    'cg19635712': 0.162209,
    'cg19668234': -0.040411,
    'cg19711783': -0.113758,
    'cg19761273': -0.108853,
    'cg19801705': 0.051839,
    'cg20011402': -0.257504,
    'cg20149168': 0.149040,
    'cg20222376': -0.106672,
    'cg20234855': 0.244570,
    'cg20386580': 0.094042,
    'cg20422417': 0.122609,
    'cg20434178': 0.341773,
    'cg20515136': -0.334444,
    'cg20559403': 0.249909,
    'cg20631204': -0.115376,
    'cg20665157': 0.099555,
    'cg20686554': 0.041199,
    'cg20744625': 0.520346,
    'cg20761322': 0.066527,
    'cg20912517': -0.185820,
    'cg21038957': -0.103066,
    'cg21159778': 0.090493,
    'cg21165519': -0.205365,
    'cg21186299': 2.029120,
    'cg21200656': 0.246572,
    'cg21213853': 0.222802,
    'cg21281007': 0.224273,
    'cg21495653': 0.222406,
    'cg21610904': -0.027103,
    'cg21824162': -0.665451,
    'cg21828345': 0.070299,
    'cg21988244': 0.031230,
    'cg22158769': 0.107564,
    'cg22266749': -0.100535,
    'cg22268271': -0.453787,
    'cg22517995': 0.354932,
    'cg22527345': 0.231810,
    'cg22639325': -0.072690,
    'cg22682373': 0.267342,
    'cg22697239': 0.176375,
    'cg22796704': -0.280375,
    'cg22849665': -0.217533,
    'cg22943590': -0.239895,
    'cg22947322': -0.052930,
    'cg23078123': -0.242301,
    'cg23104823': -0.613725,
    'cg23125200': -0.033819,
    'cg23347399': 0.013117,
    'cg23684204': 0.104843,
    'cg23715237': -0.231238,
    'cg23744638': -0.129161,
    'cg23956238': 0.533778,
    'cg24174665': 0.187453,
    'cg24350475': -0.101080,
    'cg24398933': -0.054795,
    'cg24408436': 0.156341,
    'cg24429836': -0.100258,
    'cg24489983': -0.115641,
    'cg24510518': -0.035831,
    'cg24515575': 0.267510,
    'cg24524451': -0.054761,
    'cg24667575': 0.242073,
    'cg24672014': 0.054554,
    'cg24686644': -0.508029,
    'cg24707573': 0.045543,
    'cg24871743': -0.002360,
    'cg24893035': 0.047359,
    'cg25052374': -0.015616,
    'cg25067162': 0.029022,
    'cg25108022': 0.341638,
    'cg25129541': -0.152400,
    'cg25267487': 0.047397,
    'cg25698236': -0.122299,
    'cg25711358': -0.106817,
    'cg25771195': 0.164269,
    'cg25909396': -0.275267,
    'cg26002713': -0.164871,
    'cg26116103': -0.265104,
    'cg26256771': -0.302967,
    'cg26311208': -0.022138,
    'cg26748578': -0.069656,
    'cg26765295': 0.048243,
    'cg26962778': 0.083161,
    'cg27239168': 0.270015,
    'cg27361964': -0.054397,
    'cg27651143': 0.235585,
}
intercept = -2.349778

## Load features

In [6]:
model.features = list(coefficient_dict.keys())

## Load weights into base model

In [7]:
weights = torch.tensor(list(coefficient_dict.values())).unsqueeze(0)
intercept = torch.tensor([intercept])

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = 'anti_log_linear'
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Tomusiak, Alan, et al. "Development of an epigenetic clock '
             'resistant to changes in immune cell composition." Communications '
             'Biology 7.1 (2024): 934.',
 'clock_name': 'intrinclock',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s42003-024-06609-4',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2024}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: 'anti_log_linear'
postprocess_dependencies: None
features: ['cg00054496', 'cg00055555', 'cg00088042', 'cg00107606', 'cg00359604', 'cg00448707', 'cg00454305', 'cg00480331', 'cg00491255', 'cg00579423', 'cg00583733', 'cg00588393', 'cg00590036', 'cg00593462', 'cg00716277', 'cg00734800', 'cg00751156', 'cg00753885', 'cg00790928', 'cg0079

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ -0.9125],
        [ 11.0212],
        [ 29.7538],
        [ 18.8043],
        [ -1.0000],
        [118.1667],
        [ -0.9992],
        [172.2491],
        [ -0.9376],
        [ 55.5021]], dtype=torch.float64, grad_fn=<IndexPutBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: .ipynb_checkpoints
